### Librerías

In [1]:
# Librerías
import pandas as pd
import requests
import datetime

# !pip install geopandas
import geopandas as gpd

C:\Users\34686\Downloads\Anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\34686\Downloads\Anaconda\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\34686\Downloads\Anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\34686\Downloads\Anaconda\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
<ipython-input-1-cc38fc615949>:7: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set t

Links:
* https://valencia.opendatasoft.com/explore/dataset/fgv-estacions-estaciones/export/ 

* https://valencia.opendatasoft.com/explore/dataset/estacions-rodalia-valencia-val/export/

### Lectura de datos

In [2]:
# Nombre de los ficheros
metro = "Datos transporte/fgv-estacions-estaciones.geojson"
emt = "Datos transporte/transporte-barrios.geojson"
renfe = "Datos transporte/estacions-rodalia-valencia-val.geojson"

# Lectura de datos
gdf_met = gpd.read_file(metro)
gdf_emt = gpd.read_file(emt)
gdf_ren = gpd.read_file(renfe)

### Primera visualización

In [3]:
# Metrovalencia
gdf_met.head()

,gid,codigo,nombre,tipo,linea,proximas_llegadas,proximes_llegadas,geo_point_2d,geometry
0,237118,110,Mas del Rosari,1,4,https://geoportal.valencia.es/geoportal-servic...,https://geoportal.valencia.es/geoportal-servic...,"{'lon': -0.43582499030541805, 'lat': 39.524959...",POINT (-0.43582 39.52496)
1,237200,183,La Cova,1,9,https://geoportal.valencia.es/geoportal-servic...,https://geoportal.valencia.es/geoportal-servic...,"{'lon': -0.48444443970429724, 'lat': 39.498889...",POINT (-0.48444 39.49889)
2,237215,121,Aeroport,1,"3,5",https://geoportal.valencia.es/geoportal-servic...,https://geoportal.valencia.es/geoportal-servic...,"{'lon': -0.47491943881419474, 'lat': 39.492366...",POINT (-0.47492 39.49237)
3,237216,8,Almàssera,1,3,https://geoportal.valencia.es/geoportal-servic...,https://geoportal.valencia.es/geoportal-servic...,"{'lon': -0.3542666730314796, 'lat': 39.5122566...",POINT (-0.35427 39.51226)
4,237142,15,Colón,1,"3,5,7,9",https://geoportal.valencia.es/geoportal-servic...,https://geoportal.valencia.es/geoportal-servic...,"{'lon': -0.3709277814305924, 'lat': 39.4701462...",POINT (-0.37093 39.47015)


In [4]:
# Emt
gdf_emt.head()

,geo_point_2d,geo_shape,codbarrio,nombre,coddistbar,coddistrit,codbar,transporte,stop_id,stop_name,stop_desc,x,y,zone_id,geometry
0,"{'lon': -0.38580384886348595, 'lat': 39.527643...",None,1,BENIFARAIG,171,17,171,emt,1137.0,Ferrer i Bign� - Aiacor,C FERRER Y BIGNE 21 - BENIFARAIG,724691.91,4378596.09,None,POINT (-0.38580 39.52764)
1,"{'lon': -0.3993369274208722, 'lat': 39.4910315...",None,1,BENICALAP,161,16,161,metrovlc,54.0,Beniferri,NULL,723646.02,4374498.31,A,POINT (-0.39934 39.49103)
2,"{'lon': -0.3872583065788684, 'lat': 39.4895628...",None,1,BENICALAP,161,16,161,metrovlc,96.0,Tr�nsits,NULL,724689.60,4374365.36,A,POINT (-0.38726 39.48956)
3,"{'lon': -0.39681665593963605, 'lat': 39.494434...",None,1,BENICALAP,161,16,161,metrovlc,99.0,Florista,NULL,723851.86,4374882.28,A,POINT (-0.39682 39.49443)
4,"{'lon': -0.3927782255244965, 'lat': 39.4960788...",None,1,BENICALAP,161,16,161,emt,1067.0,Equador - Pintor Matarana,AV ECUADOR EL 93 (DAVANT) - VAL�NCIA,724193.89,4375074.89,None,POINT (-0.39278 39.49608)


In [5]:
# Rodalies
gdf_ren.head()

,geo_point_2d,geo_shape,codi,descripci,latitud,longitud,direcci,geometry
0,"{'lon': -0.320388, 'lat': 39.5441373}",None,65005,ALBUIXECH,39.5441373,-0.320388,AVENIDA VIRGEN ALBUIXECH,POINT (-0.32039 39.54414)
1,"{'lon': -0.0775039, 'lat': 39.9543523}",None,65209,ALMASSORA,39.9543523,-0.0775039,CARRER ESTACIO FERROCARRIL,POINT (-0.07750 39.95435)
2,"{'lon': -0.2156704, 'lat': 39.7522591}",None,65202,ALMENARA,39.7522591,-0.2156704,CALLE ESTACION FERROCARRIL,POINT (-0.21567 39.75226)
3,"{'lon': -0.7875467, 'lat': 39.4273336}",None,66206,BUÑOL,39.4273336,-0.7875467,CALLE ESTACION FERROCARRIL,POINT (-0.78755 39.42733)
4,"{'lon': -0.4547076, 'lat': 39.1182487}",None,64103,CARCAIXENT,39.1182487,-0.4547076,GLORIETA ESTACION FERROCARRIL,POINT (-0.45471 39.11825)


### Modificación de atributos
Para poder mergearlos posteriormente y tener solo uno con los siguientes atributos:
* id
* nombre
* longitud
* latitud
* geometry
* linea
* tipo
* municipio

In [6]:
# Metrovalencia

def coordenadas_long(x):
    return x['lon']
def coordenadas_lat(x):
    return x['lat']


gdf_met['longitud'] = list(map(coordenadas_long, gdf_met.geo_point_2d))
gdf_met['latitud'] = list(map(coordenadas_lat, gdf_met.geo_point_2d))


columnas_eliminar = ['codigo', 'tipo', 'proximas_llegadas', 'proximes_llegadas', 'geo_point_2d']
gdf_met.drop(columnas_eliminar, axis=1, inplace=True)

gdf_met.rename(columns={'gid': 'id'}, inplace=True)

gdf_met['tipo'] = 'metrovalencia'
gdf_met['municipio'] = None

gdf_met.head()

,id,nombre,linea,geometry,longitud,latitud,tipo,municipio
0,237118,Mas del Rosari,4,POINT (-0.43582 39.52496),-0.435825,39.524960,metrovalencia,None
1,237200,La Cova,9,POINT (-0.48444 39.49889),-0.484444,39.498890,metrovalencia,None
2,237215,Aeroport,"3,5",POINT (-0.47492 39.49237),-0.474919,39.492367,metrovalencia,None
3,237216,Almàssera,3,POINT (-0.35427 39.51226),-0.354267,39.512257,metrovalencia,None
4,237142,Colón,"3,5,7,9",POINT (-0.37093 39.47015),-0.370928,39.470146,metrovalencia,None


In [7]:
# Emt

def coordenadas_long(x):
    if x is not None:
        return x['lon']
def coordenadas_lat(x):
    if x is not None:
        return x['lat']
    
def capitalizar(s):
    if s.find("'")!=-1:
        s = s[0].upper()+s[1]+s[2].upper()+s[3:].lower()
    else:
        s = s[0].upper() + s[1:].lower()
    return s

# Añadir columnas
gdf_emt['longitud'] = list(map(coordenadas_long, gdf_emt.geo_point_2d))
gdf_emt['latitud'] = list(map(coordenadas_lat, gdf_emt.geo_point_2d))
gdf_emt['linea'] = None

# Quitar filas
filas_eliminar = gdf_emt[gdf_emt['transporte']!='emt'].index
gdf_emt.drop(filas_eliminar, axis=0, inplace=True)

# Quitar columnas
columnas_eliminar = ['geo_point_2d', 'geo_shape', 'codbarrio', 'coddistbar', 'coddistrit', 'codbar', 'stop_desc', 'x', 'y', 'zone_id']
gdf_emt.drop(columnas_eliminar, axis=1, inplace=True)

# Renombrar columnas
gdf_emt.rename(columns={'stop_id': 'id'}, inplace=True)
gdf_emt.rename(columns={'stop_name': 'direccion'}, inplace=True)
gdf_emt.rename(columns={'transporte': 'tipo'}, inplace=True)
gdf_emt.rename(columns={'nombre': 'municipio'}, inplace=True)
gdf_emt.rename(columns={'direccion': 'nombre'}, inplace=True)

# Modificar columna
gdf_emt['municipio'] = list(map(capitalizar, gdf_emt.municipio))

gdf_emt.head()

,municipio,tipo,id,nombre,geometry,longitud,latitud,linea
0,Benifaraig,emt,1137.0,Ferrer i Bign� - Aiacor,POINT (-0.38580 39.52764),-0.385804,39.527644,None
4,Benicalap,emt,1067.0,Equador - Pintor Matarana,POINT (-0.39278 39.49608),-0.392778,39.496079,None
5,Benicalap,emt,1183.0,General Avil�s (par) - Dr. Nicasi Benlloch,POINT (-0.39146 39.48804),-0.391461,39.488037,None
6,Benicalap,emt,1319.0,Corts Valencianes - Palau de Congressos,POINT (-0.40182 39.49418),-0.401818,39.494184,None
7,Benicalap,emt,1331.0,Florista - Carrasqueta,POINT (-0.39621 39.49405),-0.396209,39.494049,None


In [8]:
# Renfe

def capitalizar(s):
    if s is not None:
        if s.find("'")!=-1:
            s = s[0].upper()+s[1]+s[2].upper()+s[3:].lower()
        else:
            s = s[0].upper() + s[1:].lower()
        return s

# Eliminar columnas
columnas_eliminar = ['geo_point_2d', 'geo_shape', 'direcci']
gdf_ren.drop(columnas_eliminar, axis=1, inplace=True)

# Renombrar columnas
gdf_ren.rename(columns={'codi': 'id'}, inplace=True)
gdf_ren.rename(columns={'descripci': 'municipio'}, inplace=True)

# Añadir columnas
gdf_ren['tipo'] = 'renfe'
gdf_ren['linea'] = None
gdf_ren['nombre'] = None

# Modificar columnas
gdf_ren['municipio'] = list(map(capitalizar, gdf_ren.municipio))


gdf_ren.head()

,id,municipio,latitud,longitud,geometry,tipo,linea,nombre
0,65005,Albuixech,39.5441373,-0.320388,POINT (-0.32039 39.54414),renfe,None,None
1,65209,Almassora,39.9543523,-0.0775039,POINT (-0.07750 39.95435),renfe,None,None
2,65202,Almenara,39.7522591,-0.2156704,POINT (-0.21567 39.75226),renfe,None,None
3,66206,Buñol,39.4273336,-0.7875467,POINT (-0.78755 39.42733),renfe,None,None
4,64103,Carcaixent,39.1182487,-0.4547076,POINT (-0.45471 39.11825),renfe,None,None


### Arreglar datos faltantes

In [9]:
# Nos aseguramos que todos tienen los mismos elementos

smet = set(gdf_met.columns)
semt = set(gdf_emt.columns)
sren = set(gdf_ren.columns)

smet&semt&sren

{'geometry',
 'id',
 'latitud',
 'linea',
 'longitud',
 'municipio',
 'nombre',
 'tipo'}

In [10]:
# gdf_met.to_file('metrovalencia.geojson', driver='GeoJSON')  
# gdf_emt.to_file('emt.geojson', driver='GeoJSON')  
# gdf_ren.to_file('renfe.geojson', driver='GeoJSON')  

In [11]:
# METROVALENCIA

# Falta municipio
gdf_met.head()

for i, m in enumerate(gdf_met['nombre']):
    print(i+1, m)
    
municipios_met = {}

def anyadir_municipio(estacion, municipio, dic=municipios_met, estaciones = set(gdf_met['nombre'])):
    if estacion in dic:
        print('YA EXISTE', estacion)
    elif estacion not in estaciones :
        print('NO EXISTE', estacion)
    else:
        dic[estacion] = municipio

1 Mas del Rosari
2 La Cova
3 Aeroport
4 Almàssera
5 Colón
6 Ayora
7 Bétera
8 Torrent Avinguda
9 Alberic
10 Carlet
11 Col·legi El Vedat
12 Moreres
13 Sant Isidre
14 Sant Miquel dels Reis
15 La Presa
16 Masia de Traver
17 Jesús
18 Patraix
19 Omet
20 Paiporta
21 Burjassot - Godella
22 Godella
23 Horta Vella
24 Alfauir
25 Parc Científic
26 Tomás y Valiente
27 Francesc Cubells
28 Grau - La Marina
29 Tossal del Rei
30 Av. del Cid
31 Llíria
32 Àngel Guimerà
33 Campament
34 Picanya
35 Albalat dels Sorells
36 Reus
37 Trànsits
38 Benicalap
39 La Carrasca
40 Dr. Lluch
41 Universitat Politècnica
42 Alboraia Peris Aragó
43 Fondo de Benaguasil
44 Gallipont - Torre del Virrei
45 Fuente del Jarro
46 L'Alcúdia
47 Sant Ramon
48 Garbí
49 Florista
50 Neptú
51 Machado
52 Roses
53 Amistat
54 Aragó
55 Castelló
56 Alginet
57 Quatre Carreres
58 Oceanogràfic
59 Palau de Congressos
60 Campus
61 Entrepins
62 Burjassot
63 Mislata
64 Túria
65 La Vallesa
66 Montortal
67 Rocafort
68 Moncada - Alfara
69 Masies
70 Russ

In [12]:
anyadir_municipio('Mas del Rosari', 'València')
anyadir_municipio('La Cova', 'Manises')
anyadir_municipio('Aeroport', 'Manises')
anyadir_municipio('Almàssera', 'Almàssera')
anyadir_municipio('Colón', 'València')
anyadir_municipio('Ayora', 'València')
anyadir_municipio('Bétera', 'Bétera')
anyadir_municipio('Torrent Avinguda', 'Torrent')
anyadir_municipio('Alberic', 'Alberic')
anyadir_municipio('Carlet', 'Carlet')
anyadir_municipio('Col·legi El Vedat', 'Torrent')
anyadir_municipio('Moreres', 'València')
anyadir_municipio('Sant Isidre', 'València')
anyadir_municipio('Sant Miquel dels Reis', 'València')
anyadir_municipio('La Presa', 'Manises')
anyadir_municipio('Masia de Traver', 'Riba-roja de Túria')
anyadir_municipio('Jesús', 'València')
anyadir_municipio('Patraix', 'València')
anyadir_municipio('Omet', 'Picassent')
anyadir_municipio('Paiporta', 'Paiporta')
anyadir_municipio('Burjassot - Godella', 'Burjassot')
anyadir_municipio('Godella', 'Godella')
anyadir_municipio('Horta Vella', 'Bétera')
anyadir_municipio('Alfauir', 'València')
anyadir_municipio('Parc Científic', 'Paterna')
anyadir_municipio('Tomás y Valiente', 'Paterna')
anyadir_municipio('Francesc Cubells', 'València')
anyadir_municipio('Grau - La Marina', 'València')
anyadir_municipio('Tossal del Rei', 'València')
anyadir_municipio('Av. del Cid', 'València')
anyadir_municipio('Llíria', 'Llíria')
anyadir_municipio('Àngel Guimerà', 'València')
anyadir_municipio('Campament', 'Paterna')
anyadir_municipio('Picanya', 'Picanya')
anyadir_municipio('Albalat dels Sorells', 'mu')
anyadir_municipio('Reus', 'València')
anyadir_municipio('Trànsits', 'València')
anyadir_municipio('Benicalap', 'València')
anyadir_municipio('La Carrasca', 'València')
anyadir_municipio('Dr. Lluch', 'mu')
anyadir_municipio('Universitat Politècnica', 'mu')
anyadir_municipio('Alboraia Peris Aragó', 'mu')
anyadir_municipio('Fondo de Benaguasil', 'mu')
anyadir_municipio('Gallipont - Torre del Virrei', 'L\'Eliana')
anyadir_municipio('Fuente del Jarro', 'mu')
anyadir_municipio('L\'Alcúdia', 'mu')
anyadir_municipio('Sant Ramon', 'mu')
anyadir_municipio('Garbí', 'mu')
anyadir_municipio('Florista', 'mu')
anyadir_municipio('Neptú', 'mu')
anyadir_municipio('Machado', 'mu')
anyadir_municipio('Roses', 'mu')
anyadir_municipio('Amistat', 'mu')
anyadir_municipio('Aragó', 'mu')
anyadir_municipio('Castelló', 'mu')
anyadir_municipio('Alginet', 'mu')
anyadir_municipio('Quatre Carreres', 'mu')
anyadir_municipio('Oceanogràfic', 'mu')
anyadir_municipio('Palau de Congressos', 'mu')
anyadir_municipio('Campus', 'mu')
anyadir_municipio('Entrepins', 'mu')
anyadir_municipio('Burjassot', 'mu')
anyadir_municipio('Mislata', 'mu')
anyadir_municipio('Túria', 'mu')
anyadir_municipio('La Vallesa', 'mu')
anyadir_municipio('Montortal', 'mu')
anyadir_municipio('Rocafort', 'mu')
anyadir_municipio('Moncada - Alfara', 'mu')
anyadir_municipio('Masies', 'mu')
anyadir_municipio('Russafa', 'mu')
anyadir_municipio('Orriols', 'mu')
anyadir_municipio('Pont de Fusta', 'mu')
anyadir_municipio('Vicent Andrés Estellés', 'mu')
anyadir_municipio('À Punt', 'mu')
anyadir_municipio('La Coma', 'mu')
anyadir_municipio('Estadi Ciutat de València', 'mu')
anyadir_municipio('Xàtiva', 'mu')
anyadir_municipio('Quart de Poblet', 'mu')
anyadir_municipio('Meliana', 'mu')
anyadir_municipio('Foios', 'mu')
anyadir_municipio('Les Carolines - Fira', 'mu')
anyadir_municipio('Canyamelar', 'mu')
anyadir_municipio('Font Almaguer', 'mu')
anyadir_municipio('Sagunt', 'mu')
anyadir_municipio('Campanar', 'mu')
anyadir_municipio('Massamagrell', 'mu')
anyadir_municipio('Massarrojos', 'mu')
anyadir_municipio('La Pobla de Farnals', 'mu')
anyadir_municipio('Facultats - Manuel Broseta', 'mu')
anyadir_municipio('El Clot', 'mu')
anyadir_municipio('Safranar', 'mu')
anyadir_municipio('Ll. Llarga - Terramelar', 'mu')
anyadir_municipio('Salt de l\'Aigua', 'mu')
anyadir_municipio('Bailén', 'mu')
anyadir_municipio('Rafelbunyol', 'mu')
anyadir_municipio('Alacant', 'mu')
anyadir_municipio('Faitanar', 'mu')
anyadir_municipio('L\'Eliana', 'mu')
anyadir_municipio('Riba-roja de Túria', 'mu')
anyadir_municipio('Montesol', 'mu')
anyadir_municipio('Picassent', 'mu')
anyadir_municipio('Torrent', 'mu')
anyadir_municipio('Ciutat Arts i Ciències - Justícia', 'mu')
anyadir_municipio('Natzaret', 'mu')
anyadir_municipio('La Granja', 'mu')
anyadir_municipio('Sant Joan', 'mu')
anyadir_municipio('Museros', 'mu')
anyadir_municipio('Alboraia Palmaret', 'mu')
anyadir_municipio('València la Vella', 'mu')
anyadir_municipio('València Sud', 'mu')
anyadir_municipio('Beniferri', 'mu')
anyadir_municipio('Pl. Espanya', 'mu')
anyadir_municipio('Benimodo', 'mu')
anyadir_municipio('Ausiàs March', 'mu')
anyadir_municipio('Espioca', 'mu')
anyadir_municipio('Seminari - CEU', 'mu')
anyadir_municipio('Amado Granell - Montolivet', 'mu')
anyadir_municipio('Tarongers - Ernest Lluch', 'mu')
anyadir_municipio('Mislata Almassil', 'mu')
anyadir_municipio('Cabanyal', 'mu')
anyadir_municipio('Nou d\'Octubre', 'mu')
anyadir_municipio('Alameda', 'mu')
anyadir_municipio('Manises', 'mu')
anyadir_municipio('Paterna', 'mu')
anyadir_municipio('Marítim', 'mu')
anyadir_municipio('Benimàmet', 'mu')
anyadir_municipio('Cantereria', 'mu')
anyadir_municipio('Benaguasil', 'mu')
anyadir_municipio('Realón', 'mu')
anyadir_municipio('Beteró', 'mu')
anyadir_municipio('Marxalenes', 'mu')
anyadir_municipio('Vicente Zaragozá', 'mu')
anyadir_municipio('Trinitat', 'mu')
anyadir_municipio('La Pobla de Vallbona', 'mu')
anyadir_municipio('Benimaclet', 'mu')
anyadir_municipio('La Canyada', 'mu')
anyadir_municipio('Santa Rita', 'mu')
anyadir_municipio('Massalavés', 'mu')
anyadir_municipio('Empalme', 'mu')
anyadir_municipio('La Cadena', 'mu')
anyadir_municipio('Platja Malva-rosa', 'mu')
anyadir_municipio('Platja les Arenes', 'mu')

In [13]:
# EMT
gdf_emt.head()

# Falta línea
# for i, n in enumerate(gdf_emt['nombre']):
#     print(i, n)
    
    
# Falta arreglar municipio
# for i, n in enumerate(gdf_emt['municipio']):
#     print(i, n)

,municipio,tipo,id,nombre,geometry,longitud,latitud,linea
0,Benifaraig,emt,1137.0,Ferrer i Bign� - Aiacor,POINT (-0.38580 39.52764),-0.385804,39.527644,None
4,Benicalap,emt,1067.0,Equador - Pintor Matarana,POINT (-0.39278 39.49608),-0.392778,39.496079,None
5,Benicalap,emt,1183.0,General Avil�s (par) - Dr. Nicasi Benlloch,POINT (-0.39146 39.48804),-0.391461,39.488037,None
6,Benicalap,emt,1319.0,Corts Valencianes - Palau de Congressos,POINT (-0.40182 39.49418),-0.401818,39.494184,None
7,Benicalap,emt,1331.0,Florista - Carrasqueta,POINT (-0.39621 39.49405),-0.396209,39.494049,None


In [14]:
# RENFE
gdf_ren.head()

# Falta nombre
for i, n in enumerate(gdf_ren['municipio']):
    print(i, n)
    
    
# Falta linea
# for i, n in enumerate(gdf_ren['municipio']):
#     print(i, n)

0 Albuixech
1 Almassora
2 Almenara
3 Buñol
4 Carcaixent
5 Chilches
6 Cullera
7 El rebollar
8 Gilet
9 La llosa
10 Les valls
11 Massalfassar
12 Segorbe-arrabal
13 Valencia-estacio del nord
14 Valencia-la font de sant lluis
15 Xeraco
16 Xirivella-alqueries
17 None
18 None
19 None
20 None
21 None
22 None
23 None
24 None
25 None
26 None
27 None
28 None
29 None
30 None
31 None
32 None
33 None
34 None
35 None
36 Aldaia
37 Alfafar-benetusser
38 Castello de la plana
39 Catarroja
40 Caudiel
41 Cheste
42 El puig
43 El romani
44 Estivella-albalat dels tarongers
45 Gandia
46 Jerica-viver
47 L'Alcudia
48 Loriguilla-reva
49 Montesa
50 Nules-villavieja
51 Platja i grau de gandia
52 Puçol
53 Requena
54 Roca-cuper
55 San antonio de requena
56 Siete aguas
57 Silla
58 Sueca
59 Tavernes de la valldigna
60 Utiel
61 Valencia sant isidre
62 Valencia-cabanyal
63 Vila-real
64 None
65 None
66 None
67 None
68 None
69 None
70 None
71 None
72 None
73 None
74 None
75 None
76 None
77 None
78 None
79 None
80 None
81 N

### Fusión

In [15]:
import geopandas as gpd

# Restablecer los índices de los GeoDataFrames para evitar índices duplicados
gdf_met.reset_index(drop=True, inplace=True)
gdf_emt.reset_index(drop=True, inplace=True)
gdf_ren.reset_index(drop=True, inplace=True)

In [36]:
gdf = pd.concat([gdf_met,gdf_emt, gdf_ren])

,id,nombre,linea,geometry,longitud,latitud,tipo,municipio
0,237118,Mas del Rosari,4,POINT (-0.43582 39.52496),-0.435825,39.52496,metrovalencia,None
1,237200,La Cova,9,POINT (-0.48444 39.49889),-0.484444,39.49889,metrovalencia,None
2,237215,Aeroport,"3,5",POINT (-0.47492 39.49237),-0.474919,39.492367,metrovalencia,None
3,237216,Almàssera,3,POINT (-0.35427 39.51226),-0.354267,39.512257,metrovalencia,None
4,237142,Colón,"3,5,7,9",POINT (-0.37093 39.47015),-0.370928,39.470146,metrovalencia,None
...,...,...,...,...,...,...,...,...
127,65000,None,None,None,-0.3771506,39.4669226,renfe,None
128,64004,None,None,None,-0.689953,38.908988,renfe,None
129,66205,None,None,None,-0.870661,39.454073,renfe,None
130,66214,None,None,None,-0.4473898,39.4597864,renfe,None


In [37]:
for i, row in gdf.iterrows():
    print(i, row.tipo, row.nombre)

0 renfe None
1 renfe None
2 renfe None
3 renfe None
4 renfe None
5 renfe None
6 renfe None
7 renfe None
8 renfe None
9 renfe None
10 renfe None
11 renfe None
12 renfe None
13 renfe None
14 renfe None
15 renfe None
16 renfe None
17 renfe None
18 renfe None
19 renfe None
20 renfe None
21 renfe None
22 renfe None
23 renfe None
24 renfe None
25 renfe None
26 renfe None
27 renfe None
28 renfe None
29 renfe None
30 renfe None
31 renfe None
32 renfe None
33 renfe None
34 renfe None
35 renfe None
36 renfe None
37 renfe None
38 renfe None
39 renfe None
40 renfe None
41 renfe None
42 renfe None
43 renfe None
44 renfe None
45 renfe None
46 renfe None
47 renfe None
48 renfe None
49 renfe None
50 renfe None
51 renfe None
52 renfe None
53 renfe None
54 renfe None
55 renfe None
56 renfe None
57 renfe None
58 renfe None
59 renfe None
60 renfe None
61 renfe None
62 renfe None
63 renfe None
64 renfe None
65 renfe None
66 renfe None
67 renfe None
68 renfe None
69 renfe None
70 renfe None
71 renfe None
72